In [43]:
from google_connection import Connect
from googleapiclient.http import MediaIoBaseDownload
import io
from pydub import AudioSegment
from pydub.playback import play
import subprocess
from tempfile import NamedTemporaryFile, TemporaryDirectory
import os

In [56]:
 def gdrive_open_file(file_id):
    # Get byte stream
    data = gdrive_open_as_bytes(file_id)
    return data

def gdrive_open_as_bytes(file_id):
    # Open arbitary file as bytestream
    request = drive_svc.files().get_media(fileId=file_id)
    stream = io.BytesIO()
    downloader = MediaIoBaseDownload(stream, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
    return stream.getvalue()

def play_song(audio_seg, player='vlc'):
    
    if player == 'vlc':
        command = "/Applications/VLC.app/Contents/MacOS/VLC"
    elif player == 'ffplay':
        command = player
    else:
        raise ValueError('Only "vlc" and "ffplay" as currently supported.')
    
    with NamedTemporaryFile("w+b", suffix=".wav") as f:
        song.export(f.name, 'wav')
        proc = subprocess.call([command, f.name])

def glue_songs(files_dict):
    glued_songs = AudioSegment.empty()
    for name, audio_seg in files_dict.items():        
        glued_songs += audio_seg
    return glued_songs

def play_files_continously(files_dict, play_with = 'vlc'):
    glued_songs=glue_songs(files_dict)
    play_song(glued_songs, player=play_with)

def play_files_as_playlist(files):
    # create the playlist
    with TemporaryDirectory() as td:
        f_names = [os.path.join(td, f'{n}') for n in files.keys()]
        print('exporting playlist to temp file ...')
        with open(os.path.join(td,'playlist.m3u'), 'w+t') as playlist:
            for name in f_names:
                song_name = name.split('/')[-1]
                audio_seg = files[song_name]
                playlist.writelines([name+'.wav\n'])
                with open(name+'.wav', 'w+b') as tempsong:
                    audio_seg.export(tempsong.name, 'wav')
        
        print('Playing on VLC ...')
        proc = subprocess.call(
            ["/Applications/VLC.app/Contents/MacOS/VLC", playlist.name]
        )
        
def to_pydub_audiosegment(drive_songs_dict):
    pass

def choose_song(parent_folder_id):
    
    songs = drive_svc.files().list(
        q = "'{folderId}' in parents".format(folderId=parent_folder_id),
        spaces='drive',
        fields='nextPageToken, files(id, name)',
        pageToken=None
    ).execute()
    
    song_formats = ['flac', 'm4a', 'mp3', 'wav']
    names = sorted([f['name'] for f in songs['files'] if f['name'].split('.')[-1] in song_formats])
    
    while True:
        choice_str = """
Choose a song from the below list:
0) Playlist with all songs
1) All album glued as one continuous song
"""
        for i, n in enumerate(names):
            choice_str += "{0}) {1}\n".format(i+2, n)
        print(choice_str)
        selection = input('Enter number of the selection: ')
        try: 
            selection = int(selection) # Try converting the input to integer.
            if selection == 0:
                # check if there is already 
                # create playlist with all songs
                return 0
            elif selection == 1:
                # glue all songs together and play it
                return 1
            elif (selection>1 and selection<len(music_folders['files'])+2): 
                # Sucessfull choice.
                print(" ")
                return songs['files'][selection-2]             
            else: 
                # Inputed a number not in the list of options.
                print("Error: Selection not in the valid list.") 
        except ValueError: 
            print("That's not a number") # If user didn't entered an integer
    
def choose_album_folder(parent_folder_id):
    
    music_folders = drive_svc.files().list(
        q = "'{folderId}' in parents".format(folderId=parent_folder_id),
        spaces='drive',
        fields='nextPageToken, files(id, name)',
        pageToken=None
    ).execute()
    
    while True:
        choice_str = """Choose a folder from the below list:\n"""
        for i, n in enumerate(music_folders['files']):
            choice_str += "{0}) {1}\n".format(i, n['name'])
        print(choice_str)
        selection = input('Enter number of the selected music folder: ')
        try: 
            selection = int(selection) # Try converting the input to integer.
            if (selection>=0 and selection<len(music_folders['files'])): 
                # Sucessfull choice.
                print(" ")
                return music_folders['files'][selection]             
            else: 
                # Inputed a number not in the list of options.
                print("Error: Selection not in the valid list.") 
        except ValueError: 
            print("That's not a number") # If user didn't entered an integer

In [14]:
drive_conn = Connect()
drive_svc = drive_conn.get_service()

In [15]:
music_folder = '1XS5yDT8Ym3ME0SUywRuMFm5AGxaSIzaN'

In [46]:
len(music_folders['files'])

3

In [59]:
album_chosen = choose_album_folder(parent_folder_id=music_folder)
song_chosen = choose_song(album_chosen['id'])

Choose a folder from the below list:
0) Opeth
1) Pink Floyd - The Wall [MFSL 1991] (1979) FLAC
2) Iron Maiden - Seventh Son of a Seventh Son

Enter number of the selected music folder: 0
 

Choose a song from the below list:
0) Playlist with all songs
1) All album glued as one continuous song
2) 01 - The Leper Affinity.flac
3) 01 - The Leper Affinity.m4a

Enter number of the selection: 2
 


In [60]:
song_chosen

{'id': '1Hd4xLYPw4m3fv73dwjWI_tIXrqqfhD4Q', 'name': 'Opeth'}

In [25]:
files_inside = drive_svc.files().list(
    q = "'{folderId}' in parents".format(folderId=choice['id']),
    #q="mimeType='image/jpeg'",
    spaces='drive',
    fields='nextPageToken, files(id, name)',
    pageToken=None
).execute()

In [26]:
#song_formats = ['flac', 'm4a', 'mp3', 'wav']
song_formats = ['m4a']
names = sorted([f['name'] for f in files_inside['files'] if f['name'].split('.')[-1] in song_formats])

In [27]:
names

['01 - The Leper Affinity.m4a']

In [28]:
files = {}
for n in names:
    for f in files_inside['files']:
        if f['name'] == n:
            
            song_name = n.split('.')[0]
            file_fmt = n.split('.')[-1]
            
            file_data = gdrive_open_file(file_id=f['id'])
            song = AudioSegment.from_file(
                io.BytesIO(file_data), 
                format=file_fmt
            )
            files[song_name] = song

In [29]:
files.keys()

dict_keys(['01 - The Leper Affinity'])

In [33]:
play_files_as_playlist(files)

exporting playlist to temp file ...
Playing on VLC ...


KeyboardInterrupt: 